openai 1.3.2 api 调用方法

In [ ]:
from openai import OpenAI
import httpx

api_key = "sk-MWCZbiYqiQUjacuGF53a6c71E3134177A585CeFe79D10aD2"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)

with open("dataset\Android\Android_2k.log", "r") as file:
    log_messages = file.readlines()[:10]


for log_message in log_messages:
    prompt = "repeat " + log_message
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content":prompt}],
    )
    print(response.choices[0].message.content)

用panda读取csv文件和log文件

In [ ]:
import pandas as pd

dataset = 'Spark'

with open('dataset\\' + dataset +
          '\\' + dataset + '_2k.log', 'r') as logfile:
    lines = logfile.readlines()

# print len(log_messages)
print(lines.__len__())

print(lines[0])

log_templates = pd.read_csv('dataset\\' + dataset + '\\' +
                            dataset + '_2k.log_structured_corrected.csv')['EventTemplate']

print(len(log_templates))

In [ ]:
s = "\n这是一\n个字符串\n"
s = s.strip('\n')
print(s)

Two Stages 

In [ ]:
from openai import OpenAI
import httpx

api_key = "sk-MWCZbiYqiQUjacuGF53a6c71E3134177A585CeFe79D10aD2"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)

logs = ['17/06/09 20:10:48 INFO broadcast.TorrentBroadcast: Reading broadcast variable 0 took 21 ms',
        '17/06/09 20:10:48 INFO storage.MemoryStore: Block broadcast_0 stored as values in memory (estimated size 384.0 B, free 317.5 KB)',
        '17/06/09 20:10:48 INFO broadcast.TorrentBroadcast: Started reading broadcast variable 4']

codes = ['''logger.info("Reading broadcast variable {} took {} ms", 0, 21)''',
         '''logger.info("Block {} stored as values in memory (estimated size {} B, free {})", "broadcast_0", 384.0, 317.5)"''',
         '''logger.info("Started reading broadcast variable {}", 4)'''
         ]

instruction = '''Given the following raw log, firstly you should generate the corresponding logging statement (The logging statement is a single line of code that is used to log a message), then you should generate the log template by identifying the variable parts and replacing them with placeholders in the form of <*>, return the code and log template without additional explainations:'''

demonstrations = [
    {"role": "user", "content": logs[1]},
    {   
        "role": "assistant",
        "content": 'Logging statement: logger.info("Block {} stored as values in memory (estimated size {}, free {})", "broadcast_0", "384.0 B", "317.5 KB")\nLog template: Block <*> stored as bytes in memory (estimated size <*>, free <*>)'
    },
]

messages = []
messages.append({"role": "system", "content": instruction})
messages.append(demonstrations[0])
messages.append(demonstrations[1])
messages.append({"role": "user", "content": '17/06/09 20:10:48 INFO storage.MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 95.0 B, free 317.6 KB)'})


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)

print(response.choices[0].message.content)  

In [ ]:
l1, l2 = response.choices[0].message.content.split('\n')
print(l1.replace('Logging statement: ',''))
print(l2.replace('Log template: ',''))

In [ ]:
from main import reverse
# choose dataset
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac',
            'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
dataset = datasets[11]

reverse('Android', 0)

In [ ]:
list = [1, 2, 3, 4, 5]
from tqdm import tqdm
import time

for i, q in enumerate(tqdm(list)):
    time.sleep(1)
    print(str(i) +':'+ str(q))

In [ ]:
demonstrations = {
    'Spark': [{"role": "user", "content": 'Log: 17/06/09 20:10:48 INFO storage.MemoryStore: Block broadcast_0 stored as values in memory (estimated size 384.0 B, free 317.5 KB)'}, {"role": "assistant", "content": 'Logging statement: logger.info("Block {} stored as values in memory (estimated size {}, free {})", "broadcast_0", "384.0 B", "317.5 KB")'}],
    'Hadoop': [{"role": "user", "content": 'Log: 2015-10-18 18:01:53,885 INFO [AsyncDispatcher event handler] org.apache.hadoop.mapreduce.v2.app.job.impl.TaskAttemptImpl: attempt_1445144423722_0020_m_000000_0 TaskAttempt Transitioned from NEW to UNASSIGNED'}, {"role": "assistant", "content": 'Logging statement: logger.info("{} TaskAttempt Transitioned from NEW to UNASSIGNED", "attempt_1445144423722_0020_m_000000_0")'}],
    'Linux': [{"role": "user", "content": 'Log: Jun 15 20:05:31 combo sshd(pam_unix)[24141]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=d211-116-254-214.rev.krline.net'}, {"role": "assistant", "content": 'Logging statement: logger.info("authentication failure; logname= uid={} euid={} tty={} ruser= rhost={}", "0", "0", "NODEVssh", "d211-116-254-214.rev.krline.net")'}],
}

print(demonstrations['Spark'][1])

In [2]:
from openai import OpenAI
from tqdm import tqdm
import httpx
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)

api_key = "sk-MWCZbiYqiQUjacuGF53a6c71E3134177A585CeFe79D10aD2"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    # http_client=httpx.Client(
    #     proxies="http://127.0.0.1:7890"  # 代理地址
    # ),
)

dataset = 'Linux'

# @retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(5))
def get_responce(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
    )
    return response.choices[0].message.content.strip('\n')


instruction = '''Given the following raw log, you should generate the corresponding logging statement. The logging statement is a single line of code that is used to log a message. You need to understand the information in the raw log, determine the log level, extract all possible variables and replace them with placeholders, and finally generate the logging statement using the correct logging function and the log message with placeholders. All extracted variables should be passed as arguments to the logging function.'''

demonstrations = {
    'Spark': [{"role": "user", "content": 'Log: 17/06/09 20:10:48 INFO storage.MemoryStore: Block broadcast_0 stored as values in memory (estimated size 384.0 B, free 317.5 KB)'}, {"role": "assistant", "content": 'Logging statement: logger.info("Block {} stored as values in memory (estimated size {}, free {})", "broadcast_0", "384.0 B", "317.5 KB")'}],
    'Hadoop': [{"role": "user", "content": 'Log: 2015-10-18 18:01:53,885 INFO [AsyncDispatcher event handler] org.apache.hadoop.mapreduce.v2.app.job.impl.TaskAttemptImpl: attempt_1445144423722_0020_m_000000_0 TaskAttempt Transitioned from NEW to UNASSIGNED'}, {"role": "assistant", "content": 'Logging statement: logger.info("{} TaskAttempt Transitioned from NEW to UNASSIGNED", "attempt_1445144423722_0020_m_000000_0")'}],
    'Linux': [{"role": "user", "content": 'Log: Jun 15 20:05:31 combo sshd(pam_unix)[24141]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=d211-116-254-214.rev.krline.net'}, {"role": "assistant", "content": 'Logging statement: logger.info("authentication failure; logname= uid={} euid={} tty={} ruser= rhost={}", "0", "0", "NODEVssh", "d211-116-254-214.rev.krline.net")'}],
}

messages = [{"role": "system", "content": instruction}]
messages.append(demonstrations[dataset][0])
messages.append(demonstrations[dataset][1])
log = 'Jun 15 02:04:59 combo sshd(pam_unix)[20898]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root'
messages.append({"role": "user", "content": log})
response = get_responce(messages)
print(response)

Logging statement: logger.info("authentication failure; logname= uid={} euid={} tty={} ruser= rhost={} user={}", "0", "0", "NODEVssh", "220-135-151-1.hinet-ip.hinet.net", "root")


Evaluate_gpt

In [1]:
from evaluator import evaluate
import pandas as pd
dataset = 'Spark'
file = f'outputs/enhanced_gpt/1shot/{dataset}.csv'


def extract_and_replace(input_string):
    # 提取第一个“”中的内容
    start = input_string.find('"') + 1
    end = input_string.find('"', start)
    extracted = input_string[start:end]
    # 将所有{}替换成<*>
    replaced = extracted.replace('{}', '<*>')
    return replaced

df = pd.read_csv(file)
df['LogTemplate'] = df['LoggingStatement'].apply(extract_and_replace)
df.to_csv(file, index=False)
evaluate(file,dataset, 1)

Precision: 1.0000, Recall: 0.9994, F1_measure: 0.9997, Group Accuracy: 0.9260, Message-Level Accuracy: 0.9925, Edit Distance: 0.1455


(0.926, 0.9925, 0.1455, 2.2247089135435227)